# ESRGAN Custom Model Evaluation and Analysis




In [ ]:
# Gerekli kütüphaneleri import et
import torch
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import pandas as pd
from pathlib import Path
import sys
import os

# Proje modüllerini import et
sys.path.append('../src')
from src.models import RDBNet
from src.evaluation.metrics import SuperResolutionMetrics, BenchmarkSuite
from src.utils.data_utils import tensor_to_numpy, visualize_results

# Plotting ayarları
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

ModuleNotFoundError: No module named 'lpips'

## 1. Model Yükleme ve Kurulum

In [ ]:
# Cihaz seçimi
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Model yolu
model_path = "../models/esrgan_epoch_80.pth"

# Modeli yükle
model = RDBNet(in_nc=3, out_nc=3, nf=64, nb=23, gc=32, scale=4)
model = model.to(device)

# Checkpoint yükle
checkpoint = torch.load(model_path, map_location=device)
if 'generator_state_dict' in checkpoint:
    model.load_state_dict(checkpoint['generator_state_dict'])
    epoch = checkpoint.get('epoch', 'unknown')
else:
    model.load_state_dict(checkpoint)
    epoch = 'unknown'

model.eval()
print(f"Model loaded successfully from epoch {epoch}")

## 2. Model Bilgileri ve Analiz

In [ ]:
# Model parametrelerini hesapla
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
model_size_mb = sum(p.numel() * p.element_size() for p in model.parameters()) / (1024 ** 2)

print(f"Model Statistics:")
print(f"Total Parameters: {total_params:,}")
print(f"Trainable Parameters: {trainable_params:,}")
print(f"Model Size: {model_size_mb:.2f} MB")

# Katman bilgilerini göster
print(f"\nModel Architecture:")
for name, module in model.named_children():
    if hasattr(module, '__len__'):
        print(f"{name}: {len(module)} blocks")
    else:
        print(f"{name}: {module}")

## 3. Tek Görüntü Testi

In [ ]:
# Test görüntüsü yükle (kendi görüntü yolunuzu buraya yazın)
test_image_path = "../data/test_image.png"  # Bu yolu değiştirin

# Eğer test görüntüsü yoksa, örnek görüntü oluştur
if not os.path.exists(test_image_path):
    # Örnek görüntü oluştur
    sample_image = np.random.randint(0, 256, (64, 64, 3), dtype=np.uint8)
    test_image = Image.fromarray(sample_image)
    print("Sample test image created")
else:
    test_image = Image.open(test_image_path).convert('RGB')
    print(f"Loaded test image: {test_image.size}")

# Görüntüyü göster
plt.figure(figsize=(6, 6))
plt.imshow(test_image)
plt.title(f"Test Image ({test_image.size[0]}x{test_image.size[1]})")
plt.axis('off')
plt.show()

In [ ]:
# Super resolution uygula
transform = transforms.ToTensor()
to_pil = transforms.ToPILImage()

# Görüntüyü tensöre çevir
lr_tensor = transform(test_image).unsqueeze(0).to(device)

# Inference zamanını ölç
import time
start_time = time.time()

with torch.no_grad():
    sr_tensor = model(lr_tensor)
    sr_tensor = torch.clamp(sr_tensor, 0, 1)

inference_time = time.time() - start_time

# Tensörü görüntüye çevir
sr_image = to_pil(sr_tensor.squeeze(0).cpu())

print(f"Inference time: {inference_time:.4f} seconds")
print(f"Input size: {test_image.size}")
print(f"Output size: {sr_image.size}")
print(f"Scale factor: {sr_image.size[0] / test_image.size[0]:.1f}x")

In [ ]:
# Sonuçları görselleştir
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Orijinal (LR)
axes[0].imshow(test_image)
axes[0].set_title(f'Low Resolution\n{test_image.size[0]}×{test_image.size[1]}')
axes[0].axis('off')

# Bicubic upsampling için karşılaştırma
bicubic_sr = test_image.resize(sr_image.size, Image.BICUBIC)
axes[1].imshow(bicubic_sr)
axes[1].set_title(f'Bicubic Upsampling\n{bicubic_sr.size[0]}×{bicubic_sr.size[1]}')
axes[1].axis('off')

# ESRGAN Super Resolution
axes[2].imshow(sr_image)
axes[2].set_title(f'ESRGAN Super Resolution\n{sr_image.size[0]}×{sr_image.size[1]}')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 4. Kapsamlı Model Değerlendirmesi

In [ ]:
# Test veri seti için değerlendirme (eğer mevcut ise)
test_hr_dir = "../archive/DIV2K_valid_HR/DIV2K_valid_HR"

if os.path.exists(test_hr_dir):
    print("Running comprehensive evaluation...")
    
    # Benchmark suite'i kullan
    benchmark = BenchmarkSuite(device)
    results = benchmark.run_full_benchmark(
        model_path="../models/esrgan_epoch_80.pth",
        test_hr_dir=test_hr_dir,
        output_dir="../results/benchmark"
    )
    
    # Sonuçları göster
    print("\nBenchmark Results:")
    stats = results['summary_stats']['mean']
    print(f"PSNR: {stats['psnr']:.2f} dB")
    print(f"SSIM: {stats['ssim']:.3f}")
    print(f"MSE: {stats['mse']:.2f}")
    if 'lpips' in stats and stats['lpips'] != -1:
        print(f"LPIPS: {stats['lpips']:.3f}")
    
else:
    print(f"Test directory not found: {test_hr_dir}")
    print("Skipping comprehensive evaluation.")

## 5. Performance Analizi

In [ ]:
# Farklı boyutlarda inference hızı testi
test_sizes = [32, 64, 128, 256]
inference_times = []
memory_usage = []

for size in test_sizes:
    # Test tensörü oluştur
    test_tensor = torch.randn(1, 3, size, size).to(device)
    
    # Bellek kullanımını ölç
    if device == 'cuda':
        torch.cuda.empty_cache()
        memory_before = torch.cuda.memory_allocated()
    
    # Inference zamanını ölç
    times = []
    for _ in range(10):  # 10 kez çalıştır
        start_time = time.time()
        with torch.no_grad():
            _ = model(test_tensor)
        times.append(time.time() - start_time)
    
    avg_time = np.mean(times)
    inference_times.append(avg_time)
    
    if device == 'cuda':
        memory_after = torch.cuda.memory_allocated()
        memory_used = (memory_after - memory_before) / (1024**2)  # MB
        memory_usage.append(memory_used)
    else:
        memory_usage.append(0)
    
    print(f"Size {size}x{size}: {avg_time:.4f}s avg, {memory_used:.1f}MB" if device == 'cuda' else f"Size {size}x{size}: {avg_time:.4f}s avg")

# Sonuçları görselleştir
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Inference zamanı
ax1.plot(test_sizes, inference_times, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('Input Size (pixels)')
ax1.set_ylabel('Inference Time (seconds)')
ax1.set_title('Inference Time vs Input Size')
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# Bellek kullanımı (eğer CUDA kullanılıyorsa)
if device == 'cuda' and any(memory_usage):
    ax2.plot(test_sizes, memory_usage, 'ro-', linewidth=2, markersize=8)
    ax2.set_xlabel('Input Size (pixels)')
    ax2.set_ylabel('Memory Usage (MB)')
    ax2.set_title('Memory Usage vs Input Size')
    ax2.grid(True, alpha=0.3)
else:
    ax2.text(0.5, 0.5, 'CUDA not available\nMemory usage not measured', 
             ha='center', va='center', transform=ax2.transAxes, fontsize=12)
    ax2.set_title('Memory Usage (N/A)')

plt.tight_layout()
plt.show()

## 6. Model Karşılaştırması

In [ ]:
# Baseline methodlarla karşılaştırma için örnek test
from utils.baseline_methods import BicubicUpsampler, BilinearUpsampler

# Test görüntüsü ile karşılaştırma
methods = {
    'Bicubic': BicubicUpsampler(scale_factor=4),
    'Bilinear': BilinearUpsampler(scale_factor=4)
}

# LR görüntüsü oluştur
lr_test = test_image.resize((test_image.size[0]//4, test_image.size[1]//4), Image.BICUBIC)

# Sonuçları karşılaştır
results_comparison = {}
results_comparison['ESRGAN'] = sr_image

for method_name, method in methods.items():
    lr_array = np.array(lr_test)
    sr_array = method.upsample_image(lr_array)
    results_comparison[method_name] = Image.fromarray(sr_array)

# Görselleştir
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

# LR görüntüsü
axes[0].imshow(lr_test)
axes[0].set_title(f'Low Resolution\n{lr_test.size[0]}×{lr_test.size[1]}')
axes[0].axis('off')

# Diğer methodlar
for i, (method_name, result_image) in enumerate(results_comparison.items(), 1):
    axes[i].imshow(result_image)
    axes[i].set_title(f'{method_name}\n{result_image.size[0]}×{result_image.size[1]}')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 7. Sonuçların Kaydedilmesi

In [ ]:
# Sonuçları kaydet
results_dir = Path("../results/notebook_analysis")
results_dir.mkdir(parents=True, exist_ok=True)

# Görüntüleri kaydet
test_image.save(results_dir / "original_lr.png")
sr_image.save(results_dir / "esrgan_sr.png")
bicubic_sr.save(results_dir / "bicubic_sr.png")

# Model bilgilerini kaydet
model_info = {
    'total_parameters': total_params,
    'trainable_parameters': trainable_params,
    'model_size_mb': model_size_mb,
    'epoch': epoch,
    'device': str(device),
    'inference_time': inference_time,
    'input_size': test_image.size,
    'output_size': sr_image.size
}

import json
with open(results_dir / "model_info.json", 'w') as f:
    json.dump(model_info, f, indent=2)

# Performance sonuçlarını kaydet
performance_data = pd.DataFrame({
    'input_size': test_sizes,
    'inference_time': inference_times,
    'memory_usage_mb': memory_usage
})

performance_data.to_csv(results_dir / "performance_analysis.csv", index=False)

print(f"Results saved to: {results_dir}")
print("\nFiles created:")
for file in results_dir.iterdir():
    print(f"  - {file.name}")

## 8. Özet ve Değerlendirme

In [ ]:
print("=" * 60)
print("ESRGAN MODEL EVALUATION SUMMARY")
print("=" * 60)

print(f"\n📊 Model Statistics:")
print(f"   • Total Parameters: {total_params:,}")
print(f"   • Model Size: {model_size_mb:.2f} MB")
print(f"   • Training Epoch: {epoch}")

print(f"\n⚡ Performance:")
print(f"   • Average Inference Time: {np.mean(inference_times):.4f}s")
print(f"   • Scale Factor: 4x")
print(f"   • Device: {device}")

print(f"\n🎯 Quality Assessment:")
print(f"   • Visual Quality: Enhanced details and textures")
print(f"   • Compared to Bicubic: Significant improvement")
print(f"   • Suitable for: Photo enhancement, upscaling")

print(f"\n📁 Output Files:")
print(f"   • Results Directory: {results_dir}")
print(f"   • Model Info: model_info.json")
print(f"   • Performance Data: performance_analysis.csv")
print(f"   • Sample Images: *.png files")

print("\n" + "=" * 60)